## This notebook is a short introduction to keras and TF 2.0

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
/
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import tensorflow as tf

import cProfile 

**解释from __future__ import ...**<br>
如果某个版本中出现了某个新的功能特性，而且这个特性和当前版本中使用的不兼容，也就是它在该版本中不是语言标准，那么我如果想要使用的话就需要从future模块导入。
比如：
#python2.7
print "Hello world"
#python3
print("Hello world") <br>
如果想在python2.7版本中使用("Hello world")形式，就需要从future模块导入。

### Basic of TF 2.0

In [2]:
tf.executing_eagerly() #可立即执行结果

True

**Run simple matrix calculations using tf**

In [3]:
x = [[2.]]
m = tf.matmul(x,x)
print("x matmul x = {}".format(m))

x matmul x = [[4.]]


tf.matmul()和tf.multiply()的区别:<br>
tf.matmul() - 矩阵乘法<br>
tf.multiply() - 两个矩阵对应元素各自相乘，a*b

**Creat Tensor**

In [4]:
a = tf.constant([[1,2],
                 [3,4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [5]:
#Broadcasting 
b = tf.add(a,1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [6]:
#element-wise multiplication
print(a*b)  

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [7]:
print(tf.matmul(a,b))

tf.Tensor(
[[10 13]
 [22 29]], shape=(2, 2), dtype=int32)


**Eager execution works perfectly with Numpy**

In [8]:
import numpy as np

In [9]:
c = np.multiply(a,b)
print(c)

[[ 2  6]
 [12 20]]


In [10]:
#Transfer a tensor to numpy array
print(a.numpy())

[[1 2]
 [3 4]]


### Computing gradients

Automatic differentiation is useful for implementing machine learning algorithms such as backpropagation for training neural networks. During eager execution, use tf.GradientTape to trace operations for computing gradients later

https://blog.csdn.net/xierhacker/article/details/53174558

In [32]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w
grad = tape.gradient(loss,w) #计算梯度
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


### Train a model

In [67]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:] #(10000,28,28)
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [68]:
x_train = tf.cast(x_train[...,tf.newaxis]/255, tf.float32), #(10000,28,28,1)
x_test = tf.cast(x_test[...,tf.newaxis]/255, tf.float32),

#y_train = y_train.astype('float32')
#y_test = y_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train,10) #转为one-hot coding
y_test = tf.keras.utils.to_categorical(y_test,10)

In [69]:
#Build the model using Sequential

mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,[3,3],activation='relu',
                          input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64,[3,3],activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation="softmax")
])

In [70]:
mnist_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)              

In [91]:
#Build the model using Model
inputs = tf.keras.Input(shape=(None,None,1),name="digits")
conv_1 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(inputs)
conv_2 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(conv_1)
ave_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_2)
outputs = tf.keras.layers.Dense(10,activation="softmax")(ave_pool) #老师原代码没有activation，loss不变
mnist_model_2 = tf.keras.Model(inputs=inputs,outputs=outputs)

In [92]:
mnist_model_2.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, None, None, 16)    160       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, None, None, 16)    2320      
_________________________________________________________________
global_average_pooling2d_6 ( (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                170       
Total params: 2,650
Trainable params: 2,650
Non-trainable params: 0
_________________________________________________________________


### Two training methods

##### Use keras fit method

In [73]:
mnist_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
                    validation_split=0.1,shuffle=True,
                   loss = tf.keras.losses.categorical_crossentropy,
                   metrics = ["accuracy"])

In [79]:
mnist_model_2.compile(loss = tf.keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [80]:
mnist_model_2.fit(x_train,y_train,batch_size=128,epochs=5)

Train on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 6s 601us/sample - loss: 8.5023 - accuracy: 0.0949
Epoch 2/5
10000/10000 [==============================] - 5s 540us/sample - loss: 8.5023 - accuracy: 0.0949
Epoch 3/5
10000/10000 [==============================] - 5s 536us/sample - loss: 8.5023 - accuracy: 0.0949
Epoch 4/5
10000/10000 [==============================] - 5s 535us/sample - loss: 8.5023 - accuracy: 0.0949
Epoch 5/5
10000/10000 [==============================] - 5s 550us/sample - loss: 8.5023 - accuracy: 0.0949


In [45]:
mnist_model.evaluate(x_test,y_test) #返回loss和acc

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[2.2548229751586915, 0.348]

In [54]:
x_train[0]

<tf.Tensor: id=2684, shape=(28, 28, 1), dtype=float32, numpy=
array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0. 

In [55]:
mnist_model.predict(x_test)

array([[0.09737968, 0.10347963, 0.10089481, ..., 0.09823937, 0.09532501,
        0.09982243],
       [0.08723605, 0.10043395, 0.10047138, ..., 0.09594887, 0.09283154,
        0.10052813],
       [0.09662443, 0.09826297, 0.10444248, ..., 0.09765065, 0.09816921,
        0.09943254],
       ...,
       [0.0921234 , 0.109471  , 0.10300416, ..., 0.08707511, 0.08860767,
        0.10119268],
       [0.10144995, 0.1018141 , 0.10426679, ..., 0.08626398, 0.09485953,
        0.10266515],
       [0.101684  , 0.09704186, 0.10440875, ..., 0.09141602, 0.09027241,
        0.1021513 ]], dtype=float32)

### Use TF 2.0 

In [93]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [94]:
dataset = tf.data.Dataset.from_tensor_slices((tf.cast(x_train[...,tf.newaxis]/255, tf.float32),
 tf.cast(y_train,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [95]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
#target为数字，用这个loss，如果为one-hot coding,用CategoricalCrossentropy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
loss_history = []

In [98]:
for epoch in range(5):
    total_loss = 0
    for (batch, (images,labels)) in enumerate (dataset):
        
        with tf.GradientTape() as tape:
            
            logits = mnist_model(images,training=True)
            loss_value = loss(labels,logits)
            total_loss += loss_value
            
        grads = tape.gradient(loss_value,mnist_model.trainable_variables)
        optimizer.apply_gradients(zip(grads,mnist_model.trainable_variables))
        
    print("Epoch {} finished,loss={}".format(epoch,total_loss/1000))

Epoch 0 finished,loss=0.5730273723602295
Epoch 1 finished,loss=0.5648807287216187
Epoch 2 finished,loss=0.5585320591926575
Epoch 3 finished,loss=0.5527334213256836
Epoch 4 finished,loss=0.548588752746582
